In [1]:
import asyncio
import time
import json
import os
import re

from agent_prompts import customer_sys_prompt, support_agent_sys_prompt
from autogen import ConversableAgent
from dotenv import load_dotenv
from itertools import islice

load_dotenv()

True

In [2]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "price": [0.000150, 0.000600],
            "cache_seed": None,
        }
    ]
}

In [3]:
def create_agent(
    name: str,
    sys_prompt: str,
    human_input_mode: str,
    termination_msg: str | None = None,
):
    return ConversableAgent(
        name=name,
        system_message=sys_prompt,
        is_termination_msg=lambda msg: termination_msg in msg["content"]
        if termination_msg
        else None,
        human_input_mode=human_input_mode,
        llm_config=llm_config,
        silent=True
    )

In [7]:
async def process_batch(queries: list[str], out_file, guidelines: str):
    """Processes a batch of queries."""
    for query in queries["customer_queries"]:
        # "RESOLVED" is passed as the termination argument.
        # this works in both cases where the LLM outputs RESOLVED or UNRESOLVED
        customer_agent = create_agent(
            "Customer",
            customer_sys_prompt.format(guidelines=guidelines),
            "NEVER",
            "RESOLVED",
        )
        support_agent = create_agent(
            "Support_Agent", support_agent_sys_prompt, "NEVER", "RESOLVED"
        )

        # remove number at the start if present
        query = re.sub(r"^\d+\.\s*", "", query)
        chat_result = await customer_agent.a_initiate_chat(support_agent, message=query)
        out_json = {
            "title": queries["title"], 
            "tone": queries["tone"],
            "style": queries["style"],
            "conversation": chat_result.chat_history,
            "length": len(chat_result.chat_history),
            "status": "UNRESOLVED" if "UNRESOLVED" in chat_result.chat_history[-1]["content"]\
                      else "RESOLVED"
        }
        json.dump(out_json, out_file, ensure_ascii=False)
        out_file.write("\n")

In [8]:
async def process_query_file(inp_file_str: str, out_file_str: str, batch_sz: int = 3):
    inp_file = open(inp_file_str, 'r')
    out_file = open(out_file_str, 'r')

    i = 1
    while True:
        query_set = list(islice(inp_file, batch_sz))
        if not batch:
            print(f"Finished generation for file: {inp_file_str}")
            break

        print(f"Processing query_set {i}.")
        tasks = [
            process_batch(
                queries=json.loads(queries)["customer_queries"],
                out_file=out_file,
                guidelines=open(f"/home/suchitg/amazon_help/leafdirs/{queries["title"]}/t.txt", "r").read()
            )
            for queries in query_set
        ]
        await asyncio.gather(*tasks)
        i += 1

In [10]:
await asyncio.run(process_query_file("../query_gen/queries.jsonl", "async_test.json"))

RuntimeError: asyncio.run() cannot be called from a running event loop